In [1]:
import logging
logging.root.handlers = []  # Jupyter messes up logging so needs a reset
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from smart_open import smart_open
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.cross_validation import train_test_split
from sklearn import linear_model
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
from nltk.corpus import stopwords
import gensim
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
from gensim.models import KeyedVectors
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

c:\users\danilo marinduque\appdata\local\programs\python\python35\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
2018-04-19 14:38:01,427 : INFO : 'pattern' package not found; tag filters are not available for English
c:\users\danilo marinduque\appdata\local\programs\python\python35\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
df = pd.read_csv('data/Shuffle_Data.csv')
df = df.dropna()
df['content'].apply(lambda x: len(x.split(' '))).sum()

434630

In [3]:
X, y = df['content'], df['tag']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state = 42)

## train own word2vec

In [4]:
def read(data):
    for i,line in data.iteritems():
        if(i%10==0):
            logging.info("read {0} cases".format(i))
        yield gensim.utils.simple_preprocess (line)

documents = list(read(X_train.to_frame()['content']))
logging.info ("Done reading data file") 

2018-04-19 14:38:02,714 : INFO : read 60 cases
2018-04-19 14:38:02,749 : INFO : read 240 cases
2018-04-19 14:38:02,753 : INFO : read 90 cases
2018-04-19 14:38:02,795 : INFO : read 170 cases
2018-04-19 14:38:02,847 : INFO : read 150 cases
2018-04-19 14:38:02,862 : INFO : read 140 cases
2018-04-19 14:38:02,865 : INFO : read 0 cases
2018-04-19 14:38:02,876 : INFO : read 100 cases
2018-04-19 14:38:02,933 : INFO : read 70 cases
2018-04-19 14:38:02,943 : INFO : read 40 cases
2018-04-19 14:38:02,952 : INFO : read 190 cases
2018-04-19 14:38:03,016 : INFO : read 230 cases
2018-04-19 14:38:03,023 : INFO : read 80 cases
2018-04-19 14:38:03,038 : INFO : read 110 cases
2018-04-19 14:38:03,134 : INFO : read 50 cases
2018-04-19 14:38:03,184 : INFO : read 160 cases
2018-04-19 14:38:03,239 : INFO : read 130 cases
2018-04-19 14:38:03,260 : INFO : read 210 cases
2018-04-19 14:38:03,276 : INFO : read 20 cases
2018-04-19 14:38:03,313 : INFO : Done reading data file


In [5]:
model = gensim.models.Word2Vec (documents, workers=10)
model.train(documents,total_examples=len(documents),epochs=20)

2018-04-19 14:38:03,325 : INFO : collecting all words and their counts
2018-04-19 14:38:03,328 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-04-19 14:38:03,395 : INFO : collected 15520 word types from a corpus of 318322 raw words and 202 sentences
2018-04-19 14:38:03,397 : INFO : Loading a fresh vocabulary
2018-04-19 14:38:03,428 : INFO : min_count=5 retains 4905 unique words (31% of original 15520, drops 10615)
2018-04-19 14:38:03,429 : INFO : min_count=5 leaves 300680 word corpus (94% of original 318322, drops 17642)
2018-04-19 14:38:03,450 : INFO : deleting the raw counts dictionary of 15520 items
2018-04-19 14:38:03,454 : INFO : sample=0.001 downsamples 47 most-common words
2018-04-19 14:38:03,456 : INFO : downsampling leaves estimated 217722 word corpus (72.4% of prior 300680)
2018-04-19 14:38:03,482 : INFO : estimated required memory for 4905 words and 100 dimensions: 6376500 bytes
2018-04-19 14:38:03,483 : INFO : resetting layer weights
2018-04-

2018-04-19 14:38:05,340 : INFO : worker thread finished; awaiting finish of 4 more threads
2018-04-19 14:38:05,340 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-04-19 14:38:05,342 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-04-19 14:38:05,345 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-04-19 14:38:05,348 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-04-19 14:38:05,349 : INFO : EPOCH - 2 : training on 318322 raw words (217631 effective words) took 0.2s, 949243 effective words/s
2018-04-19 14:38:05,525 : INFO : worker thread finished; awaiting finish of 9 more threads
2018-04-19 14:38:05,534 : INFO : worker thread finished; awaiting finish of 8 more threads
2018-04-19 14:38:05,548 : INFO : worker thread finished; awaiting finish of 7 more threads
2018-04-19 14:38:05,554 : INFO : worker thread finished; awaiting finish of 6 more threads
2018-04-19 14:38:05,560 : INFO : worker thread

2018-04-19 14:38:07,292 : INFO : worker thread finished; awaiting finish of 5 more threads
2018-04-19 14:38:07,295 : INFO : worker thread finished; awaiting finish of 4 more threads
2018-04-19 14:38:07,297 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-04-19 14:38:07,302 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-04-19 14:38:07,313 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-04-19 14:38:07,315 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-04-19 14:38:07,316 : INFO : EPOCH - 10 : training on 318322 raw words (217602 effective words) took 0.2s, 1085544 effective words/s
2018-04-19 14:38:07,475 : INFO : worker thread finished; awaiting finish of 9 more threads
2018-04-19 14:38:07,492 : INFO : worker thread finished; awaiting finish of 8 more threads
2018-04-19 14:38:07,519 : INFO : worker thread finished; awaiting finish of 7 more threads
2018-04-19 14:38:07,529 : INFO : worker thre

2018-04-19 14:38:09,085 : INFO : worker thread finished; awaiting finish of 6 more threads
2018-04-19 14:38:09,092 : INFO : worker thread finished; awaiting finish of 5 more threads
2018-04-19 14:38:09,093 : INFO : worker thread finished; awaiting finish of 4 more threads
2018-04-19 14:38:09,095 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-04-19 14:38:09,099 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-04-19 14:38:09,105 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-04-19 14:38:09,114 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-04-19 14:38:09,118 : INFO : EPOCH - 18 : training on 318322 raw words (217576 effective words) took 0.2s, 904875 effective words/s
2018-04-19 14:38:09,314 : INFO : worker thread finished; awaiting finish of 9 more threads
2018-04-19 14:38:09,332 : INFO : worker thread finished; awaiting finish of 8 more threads
2018-04-19 14:38:09,342 : INFO : worker threa

(4353969, 6366440)

In [6]:
word1 = "treachery"
model.wv.most_similar (positive=word1)

2018-04-19 14:38:09,592 : INFO : precomputing L2-norms of word weight vectors


[('evident', 0.8730087280273438),
 ('premeditation', 0.8530261516571045),
 ('cruelty', 0.7841089963912964),
 ('qualifying', 0.7767015099525452),
 ('conspiracy', 0.7738387584686279),
 ('abuse', 0.754420280456543),
 ('reasonable', 0.7497912049293518),
 ('strength', 0.7454909086227417),
 ('aggravating', 0.7349112629890442),
 ('superior', 0.7344695329666138)]

In [7]:
def word_averaging(wv, words):
    all_words, mean = set(), []
    
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in model.wv.vocab:
            mean.append(model.wv.vectors[model.wv.vocab[word].index])
            all_words.add(model.wv.vocab[word].index)

    if not mean:
        logging.warning("cannot compute similarity with no input %s", words)
        return np.zeros(wv.trainables.layer1_size,)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean


In [8]:
def  word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, review) for review in text_list ])

In [9]:
def w2v_tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text, language='english'):
        for word in nltk.word_tokenize(sent, language='english'):
            if len(word) < 2:
                continue
            tokens.append(word)
    return tokens

In [10]:
test_tokenized = X_test.to_frame().apply(lambda r: w2v_tokenize_text(r['content']), axis=1).values
train_tokenized = X_train.to_frame().apply(lambda r: w2v_tokenize_text(r['content']), axis=1).values

In [11]:
X_train_word_average = word_averaging_list(model,train_tokenized)
X_test_word_average = word_averaging_list(model,test_tokenized)

In [12]:
clf = OneVsRestClassifier(LinearSVC(C=4)) # C=1.0, max_iter=1000
score = cross_val_score(clf, X_train_word_average,  y_train.to_frame()['tag'], cv=5)
cv = np.mean(score)
print ("Accuracy using SVM with 5 cross validation : {}%".format(round(cv*100,2)))


Accuracy using SVM with 5 cross validation : 68.3%


In [13]:
clf.fit(X_train_word_average, y_train.to_frame()['tag'])

OneVsRestClassifier(estimator=LinearSVC(C=4, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
          n_jobs=1)

## testing

In [14]:
pred = clf.predict(X_test_word_average)
accuracy = clf.score(X_test_word_average,y_test.to_frame().tag)
print (accuracy)

0.8431372549019608


In [15]:
X_testArray = X_test.values
y_testArray = y_test.values

In [16]:
# row : actual :: column : predicted 
cf = confusion_matrix(y_testArray,pred)
print (cf)

[[14  3  0]
 [ 2 19  0]
 [ 1  2 10]]


In [17]:
#  total classes
from __future__ import division

In [18]:
total = 3
classes = {0:'Homicide', 1:'Parricide', 2:'Murder'}
category = 0

In [19]:
TrueP = []
FalseN = []
FalseP = []
TrueN = []

pre = 0
rec = 0

In [20]:
for category in range(total):
    TP = FN = FP = TN = 0 
    for i in range(total):
        for j in range(total):
            if i==category and j==category:
                TP += cf[i][j]
            elif i==category and j!=category:
                FN += cf[i][j]
            elif i!=category and j==category:
                FP += cf[i][j]
            else:
                TN += cf[i][j]
    print (classes.get(category))
    print (TP, FN, FP, TN)
    print ("Precision: " , TP/(TP+FP))
    print ("Recall: " , TP/(TP+FN))
    print ("\n")
    TrueP.append(TP)
    FalseN.append(FN)
    FalseP.append(FP)
    TrueN.append(TN)
    pre += (TP/(TP+FP))
    rec += TP/(TP+FN)

print ("pre: ", pre/3)
print ("rec:", rec/3)
    
# Average confusion matrix for all classes
TP = np.sum(TrueP)
FN = np.sum(FalseN)
FP = np.sum(FalseP)
TN = np.sum(TrueN)  


Homicide
14 3 3 31
Precision:  0.8235294117647058
Recall:  0.8235294117647058


Parricide
19 2 5 25
Precision:  0.7916666666666666
Recall:  0.9047619047619048


Murder
10 3 0 38
Precision:  1.0
Recall:  0.7692307692307693


pre:  0.8717320261437909
rec: 0.8325073619191267


In [21]:
# calculate f-score for each category
f1 = f1_score(y_testArray,pred, average=None)

print (f1)
print ("\n")
print (np.sum(f1)/3)

[0.82352941 0.84444444 0.86956522]


0.8458463578668182


In [22]:
# calculate f-score for all 
print ("F-score\n-----")
print ("Weighted: ", f1_score(y_testArray,pred, average='weighted'))
print ("Micro: ", f1_score(y_testArray,pred, average='micro'))
print ("Macro: ", f1_score(y_testArray,pred, average='macro'))

F-score
-----
Weighted:  0.8438761011651037
Micro:  0.8431372549019607
Macro:  0.8458463578668182


In [23]:
target_names = ['Homicide', 'Parricide', 'Murder']
print(classification_report(y_testArray, pred, target_names=target_names))

             precision    recall  f1-score   support

   Homicide       0.82      0.82      0.82        17
  Parricide       0.79      0.90      0.84        21
     Murder       1.00      0.77      0.87        13

avg / total       0.86      0.84      0.84        51



In [24]:
# classify(facts='Burlington 12-year-old Accused of killing two parents and wounding two siblings. Charged as a juvenile.')

## NEW DATA CHAROT

In [29]:
words1 = "Jose killed his wife".lower()
toks = w2v_tokenize_text(words1)
word_average = word_averaging(model,toks)
clf.predict([word_average])


array(['parricide'], dtype='<U9')

In [28]:
word1 = "the"
model.wv.most_similar (positive=word1)

[('its', 0.40056002140045166),
 ('such', 0.30158936977386475),
 ('entry', 0.2709656357765198),
 ('particular', 0.26304927468299866),
 ('subsequent', 0.2630005180835724),
 ('prison', 0.26007795333862305),
 ('this', 0.25675860047340393),
 ('assailant', 0.25575560331344604),
 ('florencios', 0.25463926792144775),
 ('subject', 0.25001856684684753)]

In [30]:
#save to pickle
import pickle

with open('categorizer_orig.pkl', 'wb') as f:
    pickle.dump(clf, f)
    
with open('tokenizer.pkl', 'wb') as f:
    pickle.dump(w2v_tokenize_text, f)
    
with open('word_average.pkl', 'wb') as f:
    pickle.dump(word_averaging, f)

In [31]:
words1 = "He killed his wife".lower()
toks = w2v_tokenize_text(words1)
word_average = word_averaging(model,toks)
clf.predict([word_average])


array(['parricide'], dtype='<U9')